# Conclusion

Untuk sementara ini, hasil yang paling optimal adalah dengan menggunakan model hanya dengan preprocessing utama.
Preprocessing yang dimaksud adalah resize menjadi 640 x 640

telah dicoba beberapa preprocerssing tambahan setelah resize:
- upscaling: belum menghasilkan output yang optimnal
- deskewing: belum menghasilkan output yang lebih baik dari proses dasar

In [12]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
import pytesseract
from PIL import Image

In [13]:
# load model & image tanpa preprocessing (hanya resize 640*640)

# 1. Muat Model (.pt)
model_path = 'YOLO Model/my_model.pt'
model = YOLO(model_path)

# 2. Lakukan Prediksi (Inferensi)
# Ganti 'path/ke/gambar/baru.jpg' dengan gambar yang ingin dideteksi
image_path = 'image_dataset/image1.jpeg' # contoh direktori dari gambar yang ingin diinfrence
image = cv2.imread(image_path)
resized_image = cv2.resize(image, (640, 640), interpolation=cv2.INTER_LINEAR)
# resized_image = deskew_image()
results = model.predict(source=resized_image, 
                        conf=0.5, # Batas kepercayaan (confidence threshold)
                        save=False # Simpan gambar hasil deteksi (opsional)
                       )

# 3. Akses Hasil (Bounding Box dan Label)
for r in results:
    # r.boxes berisi hasil deteksi
    boxes = r.boxes 
    for box in boxes:
        # Konversi koordinat ke format tensor/list
        xyxy = box.xyxy[0].tolist()  # [x1, y1, x2, y2]
        conf = box.conf[0].item()    # Confidence score
        cls = box.cls[0].item()      # Index kelas

        # Mendapatkan nama kelas
        label = model.names[int(cls)] 
        
        # Inilah hasil yang akan Anda gunakan untuk Cropping!
        # Misalnya: [100.5, 50.2, 450.8, 650.1] untuk 'nutrition fact'
        print(f"Objek: {label}\n BBox: {xyxy}\n Confidence: {conf:.2f}") 



0: 640x640 1 garam, 1 gula, 1 kalori, 1 lemak, 1 nutrition-fact, 1 protein, 1 takaran, 195.6ms
Speed: 7.1ms preprocess, 195.6ms inference, 10.2ms postprocess per image at shape (1, 3, 640, 640)
Objek: nutrition-fact
 BBox: [185.592041015625, 119.43133544921875, 533.88818359375, 382.81683349609375]
 Confidence: 0.93
Objek: takaran
 BBox: [193.88668823242188, 168.2691650390625, 333.5409851074219, 183.3692626953125]
 Confidence: 0.81
Objek: gula
 BBox: [212.49508666992188, 284.3038330078125, 395.9222106933594, 295.5023193359375]
 Confidence: 0.77
Objek: lemak
 BBox: [197.70941162109375, 223.01100158691406, 422.37994384765625, 237.96632385253906]
 Confidence: 0.76
Objek: kalori
 BBox: [192.92169189453125, 189.87918090820312, 414.37152099609375, 205.31271362304688]
 Confidence: 0.75
Objek: garam
 BBox: [207.2361602783203, 292.09173583984375, 424.96002197265625, 305.22845458984375]
 Confidence: 0.71
Objek: protein
 BBox: [200.30902099609375, 254.9383087158203, 411.47430419921875, 268.031311

In [14]:
output_crop_dir = "cropped_objects" # gambar hasil crop akan disimpan dalam direktori ini
os.makedirs(output_crop_dir, exist_ok=True)

# 3. Akses Hasil dan Lakukan Cropping
print("\n--- Proses Cropping Dimulai ---")
# Kita akan menggunakan 'resized_image' (gambar 640x640) untuk cropping 
# karena BBox yang dihasilkan oleh model sesuai dengan dimensi ini.

detection_count = 0
for r in results:
    boxes = r.boxes 
    for i, box in enumerate(boxes):
        # Mendapatkan BBox dan Label
        # [x1, y1, x2, y2]
        xyxy = box.xyxy[0].tolist()  
        conf = box.conf[0].item()    
        cls = box.cls[0].item()      
        label = model.names[int(cls)] 

        # Logika PENTING: Lewati 'nutrition-fact'
        if label == "nutrition-fact":
            print(f"Melewatkan objek '{label}' (Confidence: {conf:.2f})")
            continue

        # Konversi koordinat BBox ke integer untuk slicing cv2
        # Koordinat BBox sesuai dengan dimensi (640, 640)
        x_min = int(xyxy[0])
        y_min = int(xyxy[1])
        x_max = int(xyxy[2])
        y_max = int(xyxy[3])
        
        # --- Cropping dengan Slicing NumPy ---
        # Format Slicing: [y_min:y_max, x_min:x_max]
        cropped_segment = resized_image[y_min:y_max, x_min:x_max]

        # Periksa apakah hasil crop valid (tidak kosong)
        if cropped_segment.shape[0] > 0 and cropped_segment.shape[1] > 0:
            # Simpan gambar hasil crop
            # Nama file: label_index_conf.jpg
            output_filename = os.path.join(output_crop_dir, f"{label}_{detection_count:02d}.jpg")
            cv2.imwrite(output_filename, cropped_segment)
            
            print(f"✅ Berhasil crop '{label}' (Conf: {conf:.2f}). Disimpan di: {output_filename}")
            detection_count += 1
        else:
            print(f"❌ Gagal crop '{label}'. BBox tidak valid atau kosong.")

print(f"\nProses selesai. Total {detection_count} objek telah di-crop dan disimpan di direktori '{output_crop_dir}'.")


--- Proses Cropping Dimulai ---
Melewatkan objek 'nutrition-fact' (Confidence: 0.93)
✅ Berhasil crop 'takaran' (Conf: 0.81). Disimpan di: cropped_objects/takaran_00.jpg
✅ Berhasil crop 'gula' (Conf: 0.77). Disimpan di: cropped_objects/gula_01.jpg
✅ Berhasil crop 'lemak' (Conf: 0.76). Disimpan di: cropped_objects/lemak_02.jpg
✅ Berhasil crop 'kalori' (Conf: 0.75). Disimpan di: cropped_objects/kalori_03.jpg
✅ Berhasil crop 'garam' (Conf: 0.71). Disimpan di: cropped_objects/garam_04.jpg
✅ Berhasil crop 'protein' (Conf: 0.53). Disimpan di: cropped_objects/protein_05.jpg

Proses selesai. Total 6 objek telah di-crop dan disimpan di direktori 'cropped_objects'.


# ocr without preprocessing

In [16]:

# --- Konfigurasi ---
# Direktori tempat gambar hasil crop Anda berada
INPUT_CROP_DIR = "cropped_objects" 
# Faktor penskalaan untuk memperbesar gambar (misalnya 3x)
UPSCALE_FACTOR = 3 

# Opsional: Tentukan path Tesseract jika tidak terdeteksi oleh sistem
# Contoh Windows:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# --- Proses Utama OCR (Tanpa Preprocessing CV2) ---
print(f"Memulai proses OCR hanya dengan upscaling {UPSCALE_FACTOR}x pada gambar di '{INPUT_CROP_DIR}'...")

extracted_data = {}

for filename in os.listdir(INPUT_CROP_DIR):
    if filename.endswith((".jpg", ".jpeg", ".png")):
        
        file_path = os.path.join(INPUT_CROP_DIR, filename)
        
        # Dapatkan label objek (misalnya 'garam_01.jpg' -> 'garam')
        try:
            object_label = filename.split('_')[0] 
        except IndexError:
            object_label = "unknown"

        # 1. Muat Gambar Asli (Hasil Crop)
        img_cv2 = cv2.imread(file_path)
        
        if img_cv2 is None:
            print(f"Gagal memuat {filename}.")
            continue
            
        # 2. Upscaling (Penskalaan)
        # Penting untuk teks kecil. INTER_CUBIC menjaga ketajaman tepi.
        upscaled_img = cv2.resize(
            img_cv2, 
            None, 
            fx=UPSCALE_FACTOR, 
            fy=UPSCALE_FACTOR, 
            interpolation=cv2.INTER_CUBIC
        )
        
        # 3. Konversi cv2 (NumPy array BGR) ke PIL Image (Pytesseract)
        # Pytesseract akan menangani konversi grayscale/biner secara internal.
        img_pil = Image.fromarray(cv2.cvtColor(upscaled_img, cv2.COLOR_BGR2RGB)) # Konversi BGR ke RGB untuk PIL

        # 4. Terapkan OCR dengan Konfigurasi
        # --psm 7: Asumsi satu baris teks (ideal untuk label nutrisi)
        # --oem 3: Gunakan mesin Tesseract terbaru
        custom_config = r'--oem 3 --psm 7' 
        
        text = pytesseract.image_to_string(img_pil, config=custom_config)
        
        # Bersihkan teks
        cleaned_text = text.strip().replace('\n', ' ').replace('|', 'l') 

        extracted_data[object_label] = cleaned_text
        
        print(f"\n--- {filename} ({object_label}) ---")
        print(f"Teks Terekstrak: {cleaned_text}")

# --- Ringkasan Hasil ---
print("\n----------------------------------------------------")
print("✅ Ringkasan Data Terekstrak Akhir:")
for key, value in extracted_data.items():
    print(f"{key.capitalize():<10}: {value}")
print("----------------------------------------------------")

Memulai proses OCR hanya dengan upscaling 3x pada gambar di 'cropped_objects'...

--- gula_01.jpg (gula) ---
Teks Terekstrak: Guia §=-— 3g

--- gula_02.jpg (gula) ---
Teks Terekstrak: 

--- garam_04.jpg (garam) ---
Teks Terekstrak: . Raram (natrium) 20mg 27

--- protein_05.jpg (protein) ---
Teks Terekstrak: 4 ete eer 2 g S, 3

--- garam_01.jpg (garam) ---
Teks Terekstrak: 

--- lemak_02.jpg (lemak) ---
Teks Terekstrak: ae ere ec es 4 O5q 1%

--- kalori_03.jpg (kalori) ---
Teks Terekstrak: 

--- takaran_00.jpg (takaran) ---
Teks Terekstrak: Salian per kemasan: 1

--- karbo_00.jpg (karbo) ---
Teks Terekstrak: 

----------------------------------------------------
✅ Ringkasan Data Terekstrak Akhir:
Gula      : 
Garam     : 
Protein   : 4 ete eer 2 g S, 3
Lemak     : ae ere ec es 4 O5q 1%
Kalori    : 
Takaran   : Salian per kemasan: 1
Karbo     : 
----------------------------------------------------
